# NAIVE BAYES SPAM FILTER
CMSC 197, Problem Set 2

Desctirption:
For this problem set, we are tasked to create a spam classification model by implementing Naive Bayes Classifier.
Submitted by: John Markton M. Olarte

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn # To be use only for standardization, splitting of datasets, model evaluation and comparison
import email

## PREPROCESSING

### Split the dataset into 3 groups: (1) Training Set for "Ham", (2) Training Set for "Spam", and (3) Testing Set. Wherein, Folders 0-70:Train Set, and 71-127:Test Set.

In [2]:
import os

In [3]:
# Let us put first all email data in a dataframe
# columns: folder, file, email_message, category
# folder: folder where the email is located
# file: file name of the email
# email_message: email message
# category: 0-ham, 1-spam
df_main = pd.DataFrame(columns=["folder", "file", "email_message", "category"])
df_main

,folder,file,email_message,category


In [4]:
folder_path = "trec06/data"
folders = os.listdir(folder_path)

# Remove .DS_Store
folders = [folder for folder in folders if folder != ".DS_Store"]
folders

['000',
 '001',
 '002',
 '003',
 '004',
 '005',
 '006',
 '007',
 '008',
 '009',
 '010',
 '011',
 '012',
 '013',
 '014',
 '015',
 '016',
 '017',
 '018',
 '019',
 '020',
 '021',
 '022',
 '023',
 '024',
 '025',
 '026',
 '027',
 '028',
 '029',
 '030',
 '031',
 '032',
 '033',
 '034',
 '035',
 '036',
 '037',
 '038',
 '039',
 '040',
 '041',
 '042',
 '043',
 '044',
 '045',
 '046',
 '047',
 '048',
 '049',
 '050',
 '051',
 '052',
 '053',
 '054',
 '055',
 '056',
 '057',
 '058',
 '059',
 '060',
 '061',
 '062',
 '063',
 '064',
 '065',
 '066',
 '067',
 '068',
 '069',
 '070',
 '071',
 '072',
 '073',
 '074',
 '075',
 '076',
 '077',
 '078',
 '079',
 '080',
 '081',
 '082',
 '083',
 '084',
 '085',
 '086',
 '087',
 '088',
 '089',
 '090',
 '091',
 '092',
 '093',
 '094',
 '095',
 '096',
 '097',
 '098',
 '099',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',


In [5]:
# Use labels file to get the category of each email
labels = pd.read_csv("trec06/labels", sep=" ", header=None)
labels.columns = ["category", "file_path"]
# Change category from ham/spam to 0/1
labels["category"] = labels["category"].apply(lambda x: 0 if x == "ham" else 1)
# Remove "../data/" from file_path
labels["file_path"] = labels["file_path"].apply(lambda x: x.replace("../data/", ""))
labels

,category,file_path
0,0,000/000
1,1,000/001
2,1,000/002
3,0,000/003
4,1,000/004
...,...,...
37817,1,126/017
37818,1,126/018
37819,1,126/019
37820,1,126/020


In [6]:
# List of useless information to be removed from the email
stop_words = []
symbols = "!\\#$%-&*()[]{};:,./<>?@^_`~|\""
numbers = "0123456789"

# Use stop_words file to get the stop words
with open("stop_words.txt", "r") as f:
    stop_word = f.read().splitlines()
    stop_words = [word for word in stop_word]
stop_words

['a',
 'able',
 'about',
 'above',
 'abst',
 'accordance',
 'according',
 'accordingly',
 'across',
 'act',
 'actually',
 'added',
 'adj',
 'adopted',
 'affected',
 'affecting',
 'affects',
 'after',
 'afterwards',
 'again',
 'against',
 'ah',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'announce',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anymore',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apparently',
 'approximately',
 'are',
 'aren',
 'arent',
 'arise',
 'around',
 'as',
 'aside',
 'ask',
 'asking',
 'at',
 'auth',
 'available',
 'away',
 'awfully',
 'b',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'begin',
 'beginning',
 'beginnings',
 'begins',
 'behind',
 'being',
 'believe',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'biol',
 'both',
 'brief',
 'briefly',
 'but',
 'by',
 'c',
 'ca',
 'came',


In [7]:
# REMOVING USELESS INFORMATION

# Function to remove stop words
def remove_useless_info(message):
    # Convert to lower case, since stop words are in lower case
    # Check if the message is a list
    if type(message) != str:
        message = " ".join(message)
    message = message.lower()
    # split message into words
    words = message.split()
    # Remove stop words
    words = [word for word in words if word not in stop_words]
    # Rejoin words into a message
    message = " ".join(words)
    # Remove symbols
    message = message.translate(str.maketrans('', '', symbols))
    # Remove numbers
    message = message.translate(str.maketrans('', '', numbers))
    return message
    

In [8]:
# Function to collect messagge from parsed email
def get_message(parsed_email):
    message = ""
    # Check if the email is multipart
    if parsed_email.is_multipart():
        # Loop through the parts of the email
        for part in parsed_email.walk():
            # Check if the part is text/plain
            if part.get_content_type() == "text/plain":
                # Get the message
                message = part.get_payload()
                break
    # If the email is not multipart, just get the message
    else:
        message = parsed_email.get_payload()
    return message

In [9]:
# Test read an email
with open("trec06/data/000/001", 'r', encoding='ISO-8859-1') as e_mail:
    read_email = e_mail.read()

    parsed_email = email.message_from_string(read_email)

    message = get_message(parsed_email)
    print(message)

           
                                            
 LUXURY  WATCHES - BUY YOUR OWN ROLEX  FOR ONLY  $219!                                                        
                                               
  Rolex :: Cartier ::  Bvlgari  ::  Frank Muller  ::  Patek  Philippe :: Vacheron Constantin                   
   A.  Lange  & Sohne  ::  Audemars  Piguet  :: Jaeger-Lecoultre  ::  IWC  :: Officine  Panerai                                                    
 Breitling ::  Omega ::  Tag  Heuer                                   
                       
   Exapmle:                 
 ROLEX  Full 18K Gold  Daytona  for MEN -  only  $269!                                      
                                                 
    - Fast delivery                       
     -  The lowest prices  in the world                                                 
 -  Worldwide shipping                                                                      
                                

Let's start to read all emails from the folders

In [10]:
# Read all emails and put them in a dataframe
for folder in folders:
    # Get all files in folder
    files = os.listdir(f"{folder_path}/{folder}")
    for file in files:
        with open(f"{folder_path}/{folder}/{file}", "r", encoding='ISO-8859-1') as e_mail:
            # Read email file
            read_email = e_mail.read()
            # Parse email
            parsed_email = email.message_from_string(read_email)
            # Get message
            message = get_message(parsed_email)
            # Remove useless information
            message = remove_useless_info(message)
            # Get the category of the email based on the labels df
            category_label = labels[labels["file_path"] == f"{folder}/{file}"]["category"].values[0]
            # Concatenate the data to the df_main
            df_main = pd.concat([df_main, pd.DataFrame([[folder, file, message, category_label]], columns=["folder", "file", "email_message", "category"])], ignore_index=True)

In [11]:
df_main

,folder,file,email_message,category
0,000,000,mailing list queried weeks ago running set arc...,0
1,000,001,luxury watches buy rolex rolex cartier bvl...,1
2,000,002,academic qualifications prestigious nonacc red...,1
3,000,003,greetings all verify subscription planfans lis...,0
4,000,004,chauncey conferred luscious continued tonsillitis,1
...,...,...,...,...
37817,126,017,great news expec ted infinex ventures inc infx...,1
37818,126,018,oil sector going crazy weekly gift you kkpt th...,1
37819,126,019,httpvdtobjdocscaninfo suffering pain depressio...,1
37820,126,020,prosperous future increased money earning powe...,1


## CREATING THE FEAUTURE MATRICES


## COMPUTING THE PRIORS

## COMPUTING THE LIKELIHOOD OF EACH WORD

## CLASSIFYING THE EMAILS

## TESTING THE CLASSIFIER

## PERFORMANCE EVALUATION